In [1]:
import sys
sys.path.append('clustering')

from utils import *

In [2]:
import pandas as pd
import os
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), '2010s', 'cluster_df.parquet'))
cluster_df

data_dir:  /home/lyuzhuoqi/projects/clustering/data


,Scopus_label,movMF_label,movMF_distance,x_val,y_val,kmeans_label,kmeans_distance,skm_label,skm_distance,spectral_label,n2v_kmeans_label
VID,,,,,,,,,,,
202381698,Multidisciplinary,22,0.445886,-67.928200,15.572327,17,0.628846,20,0.444711,24,3
137773608,Multidisciplinary,22,0.590942,-68.405334,-55.633186,17,0.735654,20,0.559494,24,20
125754415,Multidisciplinary,22,0.574571,-68.448853,-55.613579,17,0.705024,20,0.550081,24,20
3880285,Multidisciplinary,23,0.618842,-68.407288,-55.634430,8,0.724859,17,0.610582,24,20
111155417,Chemistry,23,0.220853,-54.506985,-61.217068,11,0.495787,12,0.198758,14,4
...,...,...,...,...,...,...,...,...,...,...,...
2764485818,Medicine,21,0.268070,-28.969574,34.819569,18,0.542531,14,0.251015,8,18
83454320,Arts and Humanities,25,0.034777,78.609909,31.736822,13,0.251599,3,0.056642,21,5
16507453,Arts and Humanities,6,0.113656,89.206772,17.625090,13,0.307089,3,0.128660,21,5


# movMF V.S. KMeans

In [16]:
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))
method1, method2 = "Scopus", "n2v_kmeans"
min_threshold = 200
# min_threshold = 0

# 统计流量信息，按 Scopus_label 和 skm_label 分组计数
flow_data = (
    cluster_df.groupby([f"{method1}_label", f"{method2}_label"])
    .size()
    .reset_index(name="flow")
    .apply(lambda row: [row[f"{method1}_label"], str(row[f"{method2}_label"]), row["flow"]], axis=1)
    .tolist()
)
filtered_flow_data = [entry for entry in flow_data if entry[2] > min_threshold]
for entry in filtered_flow_data:
    entry[0] = method1 + "_" + str(entry[0])
    entry[1] = method2 + "_" + entry[1]
filtered_flow_data

data_dir:  /home/lyuzhuoqi/projects/clustering/data


[['Scopus_Agricultural and Biological Sciences', 'n2v_kmeans_2', 447],
 ['Scopus_Agricultural and Biological Sciences', 'n2v_kmeans_14', 526],
 ['Scopus_Arts and Humanities', 'n2v_kmeans_11', 237],
 ['Scopus_Arts and Humanities', 'n2v_kmeans_13', 451],
 ['Scopus_Arts and Humanities', 'n2v_kmeans_19', 353],
 ['Scopus_Biochemistry, Genetics and Molecular Biology', 'n2v_kmeans_20', 294],
 ['Scopus_Business, Management and Accounting', 'n2v_kmeans_19', 217],
 ['Scopus_Engineering', 'n2v_kmeans_7', 205],
 ['Scopus_Engineering', 'n2v_kmeans_24', 214],
 ['Scopus_Mathematics', 'n2v_kmeans_16', 219],
 ['Scopus_Medicine', 'n2v_kmeans_1', 472],
 ['Scopus_Medicine', 'n2v_kmeans_3', 1004],
 ['Scopus_Medicine', 'n2v_kmeans_10', 1014],
 ['Scopus_Medicine', 'n2v_kmeans_12', 310],
 ['Scopus_Medicine', 'n2v_kmeans_18', 958],
 ['Scopus_Medicine', 'n2v_kmeans_23', 420],
 ['Scopus_Psychology', 'n2v_kmeans_9', 228],
 ['Scopus_Social Sciences', 'n2v_kmeans_9', 238],
 ['Scopus_Social Sciences', 'n2v_kmeans_13

In [17]:
node_appearance_order = []
for entry in filtered_flow_data:
    if entry[0] not in node_appearance_order:
        node_appearance_order.append(entry[0])
    if entry[1] not in node_appearance_order:
        node_appearance_order.append(entry[1])
node_appearance_order

['Scopus_Agricultural and Biological Sciences',
 'n2v_kmeans_2',
 'n2v_kmeans_14',
 'Scopus_Arts and Humanities',
 'n2v_kmeans_11',
 'n2v_kmeans_13',
 'n2v_kmeans_19',
 'Scopus_Biochemistry, Genetics and Molecular Biology',
 'n2v_kmeans_20',
 'Scopus_Business, Management and Accounting',
 'Scopus_Engineering',
 'n2v_kmeans_7',
 'n2v_kmeans_24',
 'Scopus_Mathematics',
 'n2v_kmeans_16',
 'Scopus_Medicine',
 'n2v_kmeans_1',
 'n2v_kmeans_3',
 'n2v_kmeans_10',
 'n2v_kmeans_12',
 'n2v_kmeans_18',
 'n2v_kmeans_23',
 'Scopus_Psychology',
 'n2v_kmeans_9',
 'Scopus_Social Sciences',
 'n2v_kmeans_22']

In [18]:
# 生成一个字典，key为kmeans_label，value为该kmeans_label中出现次数最多的Scopus_label
label_counts = cluster_df.groupby([f'{method1}_label', 'Scopus_label']).size().unstack(fill_value=0)
method12Scopus = label_counts.idxmax(axis=1).to_dict()
method12Scopus = {f"{method1}_{k}": v for k, v in method12Scopus.items()}
method12Scopus

{'Scopus_Agricultural and Biological Sciences': 'Agricultural and Biological Sciences',
 'Scopus_Arts and Humanities': 'Arts and Humanities',
 'Scopus_Biochemistry, Genetics and Molecular Biology': 'Biochemistry, Genetics and Molecular Biology',
 'Scopus_Business, Management and Accounting': 'Business, Management and Accounting',
 'Scopus_Chemical Engineering': 'Chemical Engineering',
 'Scopus_Chemistry': 'Chemistry',
 'Scopus_Computer Science': 'Computer Science',
 'Scopus_Decision Sciences': 'Decision Sciences',
 'Scopus_Dentistry': 'Dentistry',
 'Scopus_Earth and Planetary Sciences': 'Earth and Planetary Sciences',
 'Scopus_Economics, Econometrics and Finance': 'Economics, Econometrics and Finance',
 'Scopus_Energy': 'Energy',
 'Scopus_Engineering': 'Engineering',
 'Scopus_Environmental Science': 'Environmental Science',
 'Scopus_Health Professions': 'Health Professions',
 'Scopus_Immunology and Microbiology': 'Immunology and Microbiology',
 'Scopus_Materials Science': 'Materials Sc

In [19]:
# 生成一个字典，key为kmeans_label，value为该kmeans_label中出现次数最多的Scopus_label
label_counts = cluster_df.groupby([f'{method2}_label', 'Scopus_label']).size().unstack(fill_value=0)
method22Scopus = label_counts.idxmax(axis=1).to_dict()
method22Scopus = {f"{method2}_{k}": v for k, v in method22Scopus.items()}
method22Scopus

{'n2v_kmeans_0': 'Social Sciences',
 'n2v_kmeans_1': 'Medicine',
 'n2v_kmeans_2': 'Agricultural and Biological Sciences',
 'n2v_kmeans_3': 'Medicine',
 'n2v_kmeans_4': 'Materials Science',
 'n2v_kmeans_5': 'Arts and Humanities',
 'n2v_kmeans_6': 'Computer Science',
 'n2v_kmeans_7': 'Engineering',
 'n2v_kmeans_8': 'Economics, Econometrics and Finance',
 'n2v_kmeans_9': 'Social Sciences',
 'n2v_kmeans_10': 'Medicine',
 'n2v_kmeans_11': 'Arts and Humanities',
 'n2v_kmeans_12': 'Medicine',
 'n2v_kmeans_13': 'Social Sciences',
 'n2v_kmeans_14': 'Agricultural and Biological Sciences',
 'n2v_kmeans_15': 'Mathematics',
 'n2v_kmeans_16': 'Mathematics',
 'n2v_kmeans_17': 'Engineering',
 'n2v_kmeans_18': 'Medicine',
 'n2v_kmeans_19': 'Social Sciences',
 'n2v_kmeans_20': 'Biochemistry, Genetics and Molecular Biology',
 'n2v_kmeans_21': 'Earth and Planetary Sciences',
 'n2v_kmeans_22': 'Social Sciences',
 'n2v_kmeans_23': 'Medicine',
 'n2v_kmeans_24': 'Engineering'}

In [20]:
node_color = node_appearance_order.copy()
for i, label in enumerate(node_appearance_order):
    if label in method12Scopus:
        node_color[i] = Scopus2color[method12Scopus[label]]
    elif label in method22Scopus:
        node_color[i] = Scopus2color[method22Scopus[label]]
node_color

['#000099',
 '#000099',
 '#000099',
 '#375623',
 '#375623',
 '#66FF66',
 '#66FF66',
 '#336699',
 '#336699',
 '#187402',
 '#D20000',
 '#D20000',
 '#D20000',
 '#FBFF57',
 '#FBFF57',
 '#7030A0',
 '#7030A0',
 '#7030A0',
 '#7030A0',
 '#7030A0',
 '#7030A0',
 '#7030A0',
 '#92D050',
 '#66FF66',
 '#66FF66',
 '#66FF66']

In [21]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# 自定义词映射
word_mapping = {'mechanics': 'mechanic',
                'mechanical': 'mechanic',
                'electrical': 'electric',
                'electronics': 'electric',
                'financial': 'finance',
                'political': 'politics',
                'historical': 'history',
                'computer': 'computing',
                'intelligent': 'intelligence',
                'agricultural': 'agriculture',
                'educational': 'education',
                'dental': 'dentistry',
                'archaeological': 'archaeology',
                'mathematical': 'mathematics',
                'mathematica': 'mathematics',
                'matematico': 'mathematics',
                'mathematicae': 'mathematics',
                'economic': 'economics',
                'chemical': 'chemistry',
                'geophysical': 'geophysics',
                'botanical': 'botany',
                'physical': 'physics',
                'entomological': 'entomology', 
                'entomologist': 'entomology',
                'biological': 'biology',
                'geographical': 'geography',
                'geological': 'geology',
                'geographer': 'geography',
                'cells': 'cell',
                'policy': 'politics',
                'psychological': 'psychology',
                'linguistical': 'linguistics',
                'linguistica': 'linguistics',
                'medical': 'medicine',
}

# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))
cluster_df = cluster_df.merge(pd.read_parquet(os.path.join(get_data_dir(), "MAG_venue.parquet")), left_index=True, right_index=True)

# 定义预处理函数，进行自定义词映射
def preprocess_text(text):
    words = text.lower().split()  # 将文本小写并按空格分词
    mapped_words = [word_mapping[word] if word in word_mapping else word for word in words]
    filtered_words = [word for word in mapped_words]
    return ' '.join(filtered_words)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [22]:
# 按 cluster_label 分组，将 OriginalVenue 文本合并为一个文档，并进行预处理
cluster_docs1 = (
    cluster_df.groupby(f'{method1}_label').parallel_apply(lambda x: preprocess_text(' '.join(x.OriginalVenue))).tolist()
)

In [23]:
# 自定义停用词
periodical_title_stopwords = [
    'journal', 'journals', 'proceedings', 'proceeding', 'transactions', 'transaction',
    'international', 'national', 'association', 'society', 'research',
    'review', 'science', 'sciences', 'university'
]

# 计算 TF-IDF
top_words_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english')+periodical_title_stopwords)
tfidf_matrix1 = top_words_vectorizer.fit_transform(cluster_docs1)

In [24]:
top_n = 5  # 每个聚类提取前 n 个关键词
# 获取每个聚类中最重要的关键词
top_words1 = {}
feature_names = top_words_vectorizer.get_feature_names_out()

# 确保 cluster_label 的顺序与 cluster_docs 一致
cluster_labels = sorted(cluster_df[f'{method1}_label'].unique())

for j, cluster_label in enumerate(cluster_labels):
    tfidf_scores = tfidf_matrix1[j].toarray().flatten()
    top_indices = tfidf_scores.argsort()[-top_n:][::-1]  # 获取前 n 个关键词的索引
    top_words1[f"{method1}_"+str(cluster_label)] = [feature_names[idx] for idx in top_indices]

top_words1

{'Scopus_Agricultural and Biological Sciences': ['plant',
  'food',
  'entomology',
  'agriculture',
  'ecology'],
 'Scopus_Arts and Humanities': ['history',
  'studies',
  'philosophy',
  'religion',
  'literature'],
 'Scopus_Biochemistry, Genetics and Molecular Biology': ['cell',
  'biology',
  'molecular',
  'genetics',
  'biotechnology'],
 'Scopus_Business, Management and Accounting': ['management',
  'business',
  'marketing',
  'accounting',
  'tourism'],
 'Scopus_Chemical Engineering': ['catalysis',
  'chemistry',
  'engineering',
  'technology',
  'thermal'],
 'Scopus_Chemistry': ['chemistry',
  'organic',
  'spectrometry',
  'spectroscopy',
  'analytical'],
 'Scopus_Computer Science': ['computing',
  'information',
  'systems',
  'intelligence',
  'ieee'],
 'Scopus_Decision Sciences': ['operations',
  'management',
  'logistics',
  'operational',
  'chain'],
 'Scopus_Dentistry': ['dentistry',
  'oral',
  'orthodontics',
  'maxillofacial',
  'surgery'],
 'Scopus_Earth and Plane

In [25]:
# 按 cluster_label 分组，将 OriginalVenue 文本合并为一个文档，并进行预处理
cluster_docs2 = (
    cluster_df.groupby(f'{method2}_label').parallel_apply(lambda x: preprocess_text(' '.join(x.OriginalVenue))).tolist()
)

In [26]:
# 计算 TF-IDF
top_words_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english')+periodical_title_stopwords)
tfidf_matrix2 = top_words_vectorizer.fit_transform(cluster_docs2)

In [27]:
top_n = 5  # 每个聚类提取前 n 个关键词
# 获取每个聚类中最重要的关键词
top_words2 = {}
feature_names = top_words_vectorizer.get_feature_names_out()

# 确保 cluster_label 的顺序与 cluster_docs 一致
cluster_labels = sorted(cluster_df[f'{method2}_label'].unique())

for j, cluster_label in enumerate(cluster_labels):
    tfidf_scores = tfidf_matrix2[j].toarray().flatten()
    top_indices = tfidf_scores.argsort()[-top_n:][::-1]  # 获取前 n 个关键词的索引
    top_words2[f"{method2}_"+str(cluster_label)] = [feature_names[idx] for idx in top_indices]

top_words2

{'n2v_kmeans_0': ['engineering', 'de', 'technology', 'studies', 'materials'],
 'n2v_kmeans_1': ['health', 'nursing', 'medicine', 'care', 'clinical'],
 'n2v_kmeans_2': ['plant', 'ecology', 'biology', 'environmental', 'soil'],
 'n2v_kmeans_3': ['medicine', 'clinical', 'surgery', 'clinics', 'dentistry'],
 'n2v_kmeans_4': ['materials',
  'chemistry',
  'engineering',
  'polymer',
  'technology'],
 'n2v_kmeans_5': ['de', 'studies', 'history', 'law', 'zeitschrift'],
 'n2v_kmeans_6': ['ieee', 'acm', 'computing', 'systems', 'processing'],
 'n2v_kmeans_7': ['engineering',
  'materials',
  'chemistry',
  'physics',
  'mechanic'],
 'n2v_kmeans_8': ['economics',
  'finance',
  'accounting',
  'politics',
  'statistics'],
 'n2v_kmeans_9': ['psychology',
  'education',
  'language',
  'psychiatry',
  'neuroscience'],
 'n2v_kmeans_10': ['medicine', 'health', 'clinical', 'nursing', 'surgery'],
 'n2v_kmeans_11': ['history', 'studies', 'law', 'de', 'medieval'],
 'n2v_kmeans_12': ['cancer',
  'oncology',

In [28]:
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))

# 统计流量信息，按 Scopus_label 和 skm_label 分组计数
flow_data = (
    cluster_df.groupby([f"{method1}_label", f"{method2}_label"])
    .size()
    .reset_index(name="flow")
    .apply(lambda row: [row[f"{method1}_label"], str(row[f"{method2}_label"]), row["flow"]], axis=1)
    .tolist()
)

filtered_flow_data = [entry for entry in flow_data if entry[2] > min_threshold]
for entry in filtered_flow_data:
    entry[0] = f"{method1}: "+','.join(top_words1[f"{method1}_"+str(entry[0])])
    entry[1] = f"{method2}: "+','.join(top_words2[f"{method2}_"+entry[1]])
filtered_flow_data

data_dir:  /home/lyuzhuoqi/projects/clustering/data


[['Scopus: plant,food,entomology,agriculture,ecology',
  'n2v_kmeans: plant,ecology,biology,environmental,soil',
  447],
 ['Scopus: plant,food,entomology,agriculture,ecology',
  'n2v_kmeans: chemistry,engineering,plant,materials,agriculture',
  526],
 ['Scopus: history,studies,philosophy,religion,literature',
  'n2v_kmeans: history,studies,law,de,medieval',
  237],
 ['Scopus: history,studies,philosophy,religion,literature',
  'n2v_kmeans: studies,education,de,management,economics',
  451],
 ['Scopus: history,studies,philosophy,religion,literature',
  'n2v_kmeans: education,studies,politics,history,economics',
  353],
 ['Scopus: cell,biology,molecular,genetics,biotechnology',
  'n2v_kmeans: cell,biology,microbiology,molecular,food',
  294],
 ['Scopus: management,business,marketing,accounting,tourism',
  'n2v_kmeans: education,studies,politics,history,economics',
  217],
 ['Scopus: engineering,mechanic,ieee,electric,systems',
  'n2v_kmeans: engineering,materials,chemistry,physics,mechani